<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Automatic Test Generation

In this project, we will build an automatic test generation and grading platform!
All we have to do is to provide a topic, the number of questions and the number of options for each question!
Based on this information, a suitable test is generated, presented to the user and graded automatically!

## Imports

In [1]:
import os
import openai


## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [5]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

### Tell GPT how to generate the test

We tell GPT to create a multiple choiz quiz. Hence we define the topic, the number of possible answers as well as the number of questions.
To enable automatical grading later, GPT needs to incorporate the correct answer!


In [7]:
def create_test_prompt(topic, num_questions, num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. " \
                 + f"Each question should have {num_possible_answers} options. "\
                 + f"Also include the correct answer for each question using the starting string 'Correct Answer: '."
    return prompt

In [9]:
create_test_prompt("Python", 4, 4)

"Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct Answer: '."

### OpenAI API Call
Let's use text-davinci-003 for normal text generation

In [10]:
response = openai.Completion.create(engine="text-davinci-003",
                                            prompt=create_test_prompt("Python", 4, 4),
                                            max_tokens=256,
                                            temperature=0.7)

In [11]:
response

<OpenAIObject text_completion id=cmpl-6bmkS58AMZAXGWLpqc2tRuK88uRaJ at 0x7f8949342db0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nQ1. What is the syntax for declaring a variable in Python?\nA. #variable\nB. var variable\nC. !variable\nD. var = variable\nCorrect Answer: D. var = variable\n\nQ2. What type of language is Python?\nA. Interpreted\nB. Compiled\nC. Assembly\nD. Machine\nCorrect Answer: A. Interpreted\n\nQ3. What type of loop is used when a set of instructions need to be repeated until a condition is met?\nA. For loop\nB. While loop\nC. Do-while loop\nD. If-else loop\nCorrect Answer: B. While loop\n\nQ4. What is the result of the following expression?\n2 + 5 * 3\nA. 23\nB. 17\nC. 11\nD. 25\nCorrect Answer: B. 17"
    }
  ],
  "created": 1674463544,
  "id": "cmpl-6bmkS58AMZAXGWLpqc2tRuK88uRaJ",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 185,
   

In [12]:
response["choices"][0]["text"]

'\n\nQ1. What is the syntax for declaring a variable in Python?\nA. #variable\nB. var variable\nC. !variable\nD. var = variable\nCorrect Answer: D. var = variable\n\nQ2. What type of language is Python?\nA. Interpreted\nB. Compiled\nC. Assembly\nD. Machine\nCorrect Answer: A. Interpreted\n\nQ3. What type of loop is used when a set of instructions need to be repeated until a condition is met?\nA. For loop\nB. While loop\nC. Do-while loop\nD. If-else loop\nCorrect Answer: B. While loop\n\nQ4. What is the result of the following expression?\n2 + 5 * 3\nA. 23\nB. 17\nC. 11\nD. 25\nCorrect Answer: B. 17'

### Q/A Extraction

We now need to extract the questions and answers to present them to the students later

In [13]:
def create_student_view(test, num_questions):
    student_view = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line+"\n"
        else:

            if question_number < num_questions:
                question_number+=1
                student_view[question_number] = ""
    return student_view
 

In [14]:
create_student_view(response["choices"][0]["text"], 4)

{1: '\n\nQ1. What is the syntax for declaring a variable in Python?\nA. #variable\nB. var variable\nC. !variable\nD. var = variable\n',
 2: '\nQ2. What type of language is Python?\nA. Interpreted\nB. Compiled\nC. Assembly\nD. Machine\n',
 3: '\nQ3. What type of loop is used when a set of instructions need to be repeated until a condition is met?\nA. For loop\nB. While loop\nC. Do-while loop\nD. If-else loop\n',
 4: '\nQ4. What is the result of the following expression?\n2 + 5 * 3\nA. 23\nB. 17\nC. 11\nD. 25\n'}

In [16]:
def extract_answers(test, num_questions):
    answers = {1 : ""}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line+"\n"

            if question_number < num_questions:
                question_number+=1
                answers[question_number] = ""
    return answers



In [17]:
extract_answers(response["choices"][0]["text"], 4)

{1: 'Correct Answer: D. var = variable\n',
 2: 'Correct Answer: A. Interpreted\n',
 3: 'Correct Answer: B. While loop\n',
 4: 'Correct Answer: B. 17\n'}

### Exam simulation
Based on the extracted questions, we can now simulate the exam

In [18]:
def take(student_view):
    answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        answers[question] = answer
    return answers


In [22]:
student_answers = take(create_student_view(response["choices"][0]["text"], 4))



Q1. What is the syntax for declaring a variable in Python?
A. #variable
B. var variable
C. !variable
D. var = variable

Enter your answer: D

Q2. What type of language is Python?
A. Interpreted
B. Compiled
C. Assembly
D. Machine

Enter your answer: A

Q3. What type of loop is used when a set of instructions need to be repeated until a condition is met?
A. For loop
B. While loop
C. Do-while loop
D. If-else loop

Enter your answer: b

Q4. What is the result of the following expression?
2 + 5 * 3
A. 23
B. 17
C. 11
D. 25

Enter your answer: b


### Automatic Grading
Based on the student's answers and correct answers, we can now grade the test!

In [41]:
def grade(correct_answer_dict, answers):
    correct_answers = 0
    for question, answer in answers.items():
        if answer.upper() == correct_answer_dict[question].upper()[16]:
            correct_answers+=1
    return 100 * correct_answers / len(correct_answer_dict)


In [42]:
grade(extract_answers(response["choices"][0]["text"], 4), student_answers)

100.0